In [45]:
import pandas as pd
import numpy as np
from collections import Counter

In [57]:
prediction_files = [
        "output/kfold_output_0_2156.csv",
        "output/kfold_output_0_4312.csv",
        "output/kfold_output_0_6468.csv",
]

In [58]:
predictions = []
for path in prediction_files:
    df = pd.read_csv(path)
    predictions.append(df)
    print(f"Loaded: {path}, Shape: {df.shape}")

Loaded: output/kfold_output_0_2156.csv, Shape: (869, 7)
Loaded: output/kfold_output_0_4312.csv, Shape: (869, 7)
Loaded: output/kfold_output_0_6468.csv, Shape: (869, 7)


In [59]:
len(predictions)

3

# soft voting (확률값 평균)

In [60]:
first_df = predictions[0]
result_df = pd.DataFrame()
result_df['id'] = first_df['id']
final_answers = []
final_probs = []

for idx, row_id in enumerate(result_df['id']):
    current_probs = []
    for pred_df in predictions:
        pred_row = pred_df[pred_df['id'] == row_id].iloc[0]
        probs = [pred_row[col] for col in pred_row.index if col.startswith('prob_')]
        probs = [0 if pd.isna(p) else p for p in probs]
        current_probs.append(probs)

    avg_probs = np.mean(current_probs, axis=0)
    answer = str(np.argmax(avg_probs) + 1)
        
    final_answers.append(answer)
    final_probs.append(avg_probs)

In [61]:
result_df['ensemble_answer'] = final_answers
result_df.to_csv("output/ensemble_result.csv")

In [62]:
result_with_prob = result_df.copy()
for i in range(len(final_probs[0])):
    result_with_prob[f'avg_prob_{i+1}'] = [probs[i] for probs in final_probs]

In [63]:
result_with_prob

,id,ensemble_answer,avg_prob_1,avg_prob_2,avg_prob_3,avg_prob_4,avg_prob_5
0,generation-for-nlp-0,1,0.938469,0.033705,0.010716,0.012401,4.709603e-03
1,generation-for-nlp-1,1,0.987593,0.005773,0.005843,0.000744,4.707602e-05
2,generation-for-nlp-2,4,0.000200,0.000193,0.000065,0.999540,2.419700e-06
3,generation-for-nlp-3,5,0.092411,0.058411,0.006956,0.013506,8.287144e-01
4,generation-for-nlp-4,3,0.229734,0.180046,0.429324,0.157940,2.955932e-03
...,...,...,...,...,...,...,...
864,generation-for-nlp-1609,1,0.999860,0.000110,0.000017,0.000010,2.381626e-06
865,generation-for-nlp-1512,1,0.999890,0.000100,0.000005,0.000004,6.733329e-07
866,generation-for-nlp-1382,3,0.000040,0.000063,0.999739,0.000154,3.834223e-06
867,generation-for-nlp-702,4,0.002178,0.000527,0.041955,0.955340,0.000000e+00


# Hard Voting (다수결)

In [ ]:
first_df = predictions[0]
    result_df = pd.DataFrame()
    result_df['id'] = first_df['id']
    
    final_answers = []
    for idx in range(len(first_df)):
        # 각 모델의 예측값 수집
        votes = [pred_df.iloc[idx]['answer'] for pred_df in predictions]
        # 최빈값 계산
        most_common = Counter(votes).most_common(1)[0][0]
        final_answers.append(most_common)
    
    result_df['answer'] = final_answers